# Create aspirational and real-world corpora

In [1]:
import os
import pandas as pd

# Load metadata

fandoms = [
    'allmarvel',
    'supernatural',
    'harrypotter',
    'dcu',
    'sherlock',
    'teenwolf',
    'starwars',
]

metadata = {}

for fandom in fandoms:
    metadata_path = f'/usr2/scratch/fanfic/ao3_{fandom}_text/stories.csv'
    metadata[fandom] = pd.read_csv(metadata_path)

In [4]:
# Tag separation

tag_corpora = {}
tag_corpora['aspirational'] = [
    "lgbtq themes",
    "coming out",
    "gay",
    "trans character",
    "gay character",
    "bi character",
    "lesbian character",
]

tag_corpora['challenges'] = [
    "homophobia",
    "slurs",
    "transphobia",
    "biphobia",
    "closeted character",
]

In [ ]:
# Filter to fics that have each
from tqdm import tqdm_notebook as tqdm

fic_ids = {corpus: {fandom: [] for fandom in fandoms} for corpus in tag_corpora}
total_fics = 0

for fandom in tqdm(fandoms):
    print(fandom)
    for corpus, tag_list in list(tag_corpora.items()):
        selected = metadata[fandom].loc[
#             [metadata[fandom]['additional tags'].map(lambda x: any([t in eval(x.lower()) for t in tag_list]))],
#             'fic_id'].tolist()
            metadata[fandom]['additional tags'].map(lambda x: any([t in eval(x.lower()) for t in tag_list])),
            'fic_id'].tolist()
        fic_ids[corpus][fandom].extend(selected) 
        
    total_fics += len(metadata[fandom])
   

In [19]:
for corpus in tag_corpora:
    print(f'{corpus}: {sum(len(fic_ids[corpus][f]) for f in fandoms)}')
print(f'Total fics: {total_fics}')

aspirational: 7300
challenges: 3765
Total fics: 1036031


In [24]:
fic_ids['challenges']['harrypotter'][:100]

[10014671,
 10049126,
 10065767,
 100802,
 1011815,
 10170614,
 1019982,
 1023267,
 1032687,
 10333220,
 10343364,
 10375506,
 10394706,
 10397595,
 10428813,
 1045248,
 10465140,
 10483320,
 1051028,
 10516722,
 1059218,
 10617489,
 1068359,
 10685394,
 1076711,
 10834797,
 10841805,
 10847154,
 10848570,
 10875816,
 10903839,
 10945338,
 1097794,
 11016537,
 110174,
 11064498,
 11065440,
 1109283,
 11100546,
 11110515,
 1111793,
 1111881,
 1113550,
 1115479,
 11170662,
 1117998,
 1118039,
 11206140,
 11228877,
 11285787,
 11291187,
 11302176,
 11302554,
 11326149,
 11329692,
 11370783,
 113801,
 11444625,
 11461320,
 11461821,
 1150239,
 1152427,
 11529567,
 11536026,
 11568321,
 1158012,
 11586813,
 11591313,
 11639019,
 11653164,
 11670942,
 11688195,
 11716779,
 11742867,
 1181013,
 11821773,
 11850354,
 11854263,
 11901132,
 11901780,
 11903853,
 11917146,
 11968704,
 11971440,
 1201150,
 12059778,
 12100377,
 12128475,
 12144147,
 12160410,
 12164061,
 12176877,
 12185310,
 1219

## Concatenate text files

In [28]:
outpath = '/usr0/home/mamille2/new_home/fanfiction/data/{}_seven_fandoms.txt'

for corpus in tag_corpora:
    
    with open(outpath.format(corpus), 'w') as outfile:
    
        for fandom in tqdm(fandoms):
            
            if fandom == 'harrypotter':
                inpath = '/usr2/mamille2/fanfiction-project/data/ao3/{0}/fics_paras/{1}_tokenized_paras.txt'
                
            else:
                inpath = '/usr2/mamille2/fanfiction-project/data/ao3/{0}/fics_paras/{1}.txt'
                
            for fic_id in tqdm(fic_ids[corpus][fandom]):

                # Load file, write file
                with open(inpath.format(fandom, fic_id), 'r') as f:
                    file = f.read()
                outfile.write(f"{file}\n")

# Check how many fics have only slash pairings with/without identity terms

In [1]:
import os
import pandas as pd

# Load metadata

fandoms = [
    'allmarvel',
    'supernatural',
    'harrypotter',
    'dcu',
    'sherlock',
    'teenwolf',
    'starwars',
]

metadata = {}

for fandom in fandoms:
    metadata_path = f'/usr2/scratch/fanfic/ao3_{fandom}_text/stories.csv'
    metadata[fandom] = pd.read_csv(metadata_path)

In [4]:
metadata['allmarvel']['category'].sample(100)

223501                  []
137093             ["M/M"]
138686           ["Multi"]
4178               ["M/M"]
173208      ["F/M", "Gen"]
233452             ["M/M"]
88342              ["M/M"]
13236              ["F/M"]
114411             ["M/M"]
178779             ["M/M"]
9978                    []
243303    ["F/F", "Multi"]
91401                   []
8969               ["F/M"]
149064    ["M/M", "Multi"]
35493       ["Gen", "F/M"]
20988              ["F/M"]
133452           ["Multi"]
212984             ["M/M"]
88045              ["M/M"]
220966             ["F/M"]
226489      ["M/M", "F/M"]
158693             ["M/M"]
119900             ["F/M"]
120006                  []
165325             ["F/F"]
168175             ["M/M"]
172387             ["Gen"]
122069             ["F/M"]
79345              ["F/M"]
                ...       
59395              ["F/M"]
223752      ["F/M", "Gen"]
64540              ["F/M"]
54522              ["M/M"]
234442             ["M/M"]
21681              ["F/F"]
2

In [9]:
for fandom in fandoms:
    
    metadata[fandom]['only_slash'] = metadata[fandom]['category'].map(
        lambda x: all(cat in ['F/F', 'M/M'] for cat in eval(x)))

In [11]:
lines = []

for fandom in fandoms:
    n_only_slash = sum(metadata[fandom]['only_slash'])
    lines.append([fandom, n_only_slash, len(metadata[fandom])-n_only_slash])
    
pd.DataFrame(lines, columns=['fandom', 'num_only_slash', 'num_other'])

,fandom,num_only_slash,num_other
0,allmarvel,115231,129969
1,supernatural,112443,77252
2,harrypotter,86690,92717
3,dcu,62764,60837
4,sherlock,68312,49288
5,teenwolf,63743,34576
6,starwars,31272,50937


In [ ]:
# Check for LGBTQ identity terms in those fics
import re

marked_labels = ['transgender', 'trans', 'queer', 'lesbian', 'gay', 'homosexual']
# unmarked_labels = ['heterosexual', 'cisgender', 'cis']

